# Lloyd W Jones III 
## Week 12 Homework part 1

- **ESearch**: Used to search for articles and returns a list of article IDs (PMIDs)
- **EFetch**: Used to retrieve detailed information about specific articles using their PMIDs

In [6]:
# Libraries 
import requests
import json
import xml.etree.ElementTree as ET
import pandas as pd 

##### url = https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=machine+learning+in+healthcare+AND+(2020/01/01[Date+-+Publication]+:+2025/04/07[Date+-+Publication])&retmax=1000&retmode=json&sort=relevance
* db = Specifies the database being searched 
* term = Search Query 
* retmax = maximum number of returns 
* retmod = format of the returns 
* sort = sorting method 

In [34]:
def ncbi_search():
    # Standard Inputs
    sort = 'relevance'
    retmode = 'json'
    baseURL = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    database = 'pubmed'
    minDate = '2020'
    maxDate = '2025'
    # User Inputs 
    
    ## Search term error handling loop
    loop = True
    while loop == True:
        term = input('What would you like to search on NCBI PubMed? ').replace(' ','+')
        checkterm = input(f'Your input has been formated for the call,\n{term} \ndoes your search look correct? [y/n] ')
        if checkterm == 'y':
            print('search input confirmed')
            loop = False
        else:
            print('answer to correct search was no or unclear please make sure to check your inputs!')
    
    ## Max Returns error handling loop
    loop = True
    while loop == True:
        try:
            retMax = int(input('What is the max number of returns you want? please input an integer. '))
            
            print(f'Your input has been formated for the call,\n{retMax}')
            checkMax = input(f'does your search look correct? [y/n] ')
        
            if checkMax == 'y':
                print('max returns input confirmed')
                loop = False
            else:
                print('answer to correct search was no or unclear please make sure to check your inputs!')
        except ValueError:
            print(f'your input was not an integer, please try again')
    
    userparams = {
                  'db': database,
                  'term': term,
                  'retmax': retMax,
                  'retmode': retmode,
                  'mindate': minDate,
                  'maxdate': maxDate,
                  'sort': sort
                 }
    ## API Call 
    
    apiResponse = requests.get(baseURL, params= userparams)
    dataAPI = apiResponse.json()
    return dataAPI

In [35]:
def process_json(jsonVar, fileName):
    with open(f'{fileName}.json', 'w') as newfile:
        json.dump(jsonVar, newfile, indent= 2)
    print('JSON File Has been generated, file name is, \nQuestion1a.json')


In [36]:
def question1A():
    dataAPI = ncbi_search()
    process_json(dataAPI,'Question1a')

In [37]:
question1A()

search input confirmed
Your input has been formated for the call,
20
max returns input confirmed
JSON File Has been generated, file name is, 
Question1a.json


In [11]:
def abstract_search():
    # Standard Inputs
    retmode = 'xml'
    baseURL = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    database = 'pubmed'
    rettype = 'abstract'
    # User Inputs 
    
    ## Search term error handling loop
    loop = True
    while loop == True:
        idInput = input('What would you like to search on NCBI PubMed? ')
        checkID = input(f'Your input has been formated for the call,\n{idInput} \ndoes your search look correct? [y/n] ')
        if checkID == 'y':
            print('search input confirmed')
            loop = False
        else:
            print('answer to correct search was no or unclear please make sure to check your inputs!')
    
    userparams = {
        'db' : database,
        'id' : idInput,
        'rettype' : rettype,
        'retmode' : retmode
        }

    loop = True
    while loop == True:
        try:
            apiResponse = requests.get(baseURL, params= userparams)
            apiResponse.raise_for_status()
            xmlAPI = apiResponse.content
            print('Search Completed Successfully')
            loop = False

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            print(f'try new id, original input was {idInput}')
            loopcheck2 = True
            while loopcheck2 == True:
                idInput = input('What would you like to search on NCBI PubMed? ')
                checkID = input(f'Your input has been formated for the call,\n{idInput} \ndoes your search look correct? [y/n] ')
                if checkID == 'y':
                    print('search input confirmed')
                    loopcheck2 = False
                else:
                    print('answer to correct search was no or unclear please make sure to check your inputs!')
            userparams = {
                'db' : database,
                'id' : idInput,
                'rettype' : rettype,
                'retmode' : retmode
                }
    
    return xmlAPI 

In [30]:
def xml_parse(xmlAPI):
    root = ET.fromstring(xmlAPI)
    print(root)
    # Parsing for wanted Information
    pmID = [elem.text for elem in root.findall('.//PMID')]
    title = [elem.text for elem in root.findall('.//ArticleTitle')]
    abstract = [elem.text for elem in root.findall('.//AbstractText')]
    date = [elem.text for elem in root.findall('.//PubDate')]
    journal = [elem.text for elem in root.findall('.//Journal/Title')]
    # Dataframe generation 
    parsedict = {
        'title' : title,
        'abstract' : abstract,
        'date' : date,
        'journal' : journal
    }

    return parsedict

In [38]:
def question1BC():
    xmlAPI = abstract_search()
    output = xml_parse(xmlAPI)
    return output


In [39]:
question1BC()

search input confirmed
Search Completed Successfully
<Element 'PubmedArticleSet' at 0x1507a91c0>


{'title': ['Cancer and cure: A critical analysis.'],
 'abstract': ['Cancer is one of the most dreaded diseases of the 20th century and spreading further with continuance and increasing incidence in the 21st century. The situation is so alarming that every fourth person is having a lifetime risk of cancer. India registers more than 11 lakh new cases of cancer every year, whereas, this figure is above 14 million worldwide. Is cancer curable? The short answer to this question is "Yes." In fact, all cancers are curable if they are caught early enough. Cancer cells continue to grow unless one of four things occur: (1) The cancerous mass is removed surgically; (2) using chemotherapy or another type of cancer-specific medication, such as hormonal therapy; (3) using radiation therapy; or (4) the cancer cells shrink and disappear on their own.'],
 'date': [None],
 'journal': ['Indian journal of cancer']}

# Reflection
* This homework project made me do a lot of research and call back to skills I haven't worked on since the CompSci Class last year
* I implemented some very basic error handling for my user inputs as well as for my exception handling 
* For some parts of this project I used Copilot to help me understand what was causing my errors when using my api calls
    * biggest issue was using the wrong base url for question 1B 
* all in all this homework project was informative, It would have been more ideal if we had a bit more time to complete it.